In [ ]:
The landsat_hires netcdf file can be found at http://glasslab.engr.ccny.cuny.edu/u/hannah/eas213/landsat_hires.nc

If you have the file on your computer, set the path to the location on disk:

```python
datapath = "/path/to/landsat_hires.nc"
```
if you are running the file via mybinder (the interactive version), then set the path to the managed version of the data:

```python
from quilt.data.story645.eas213 import landsat_hires
datapath = landsat_hires._data()
```

You can also install quilt and the data package on your computer to access the data. To do so, enter the following commands in a terminal or anaconda command prompt:

```bash
pip install quilt
quilt install story645/eas213
```
